<a href="https://colab.research.google.com/github/robimalco/colab/blob/main/Home_Credit_Default_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# START SETUP

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files

In [ ]:
files.upload()

In [ ]:
# !mkdir ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!rm -rf sample_data

In [ ]:
!kaggle competitions download -c home-credit-default-risk

In [ ]:
!unzip POS_CASH_balance.csv.zip
!unzip application_test.csv.zip
!unzip application_train.csv.zip
!unzip bureau.csv.zip
!unzip bureau_balance.csv.zip
!unzip credit_card_balance.csv.zip
!unzip installments_payments.csv.zip
!unzip previous_application.csv.zip

In [ ]:
!rm -rf *.zip

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)

# Files

*   application_{train|test}.csv --> main table, static data for all applications. One row represents one loan in our data sample.
*   bureau.csv -->  client's previous credits, for every loan in our sample, there are as many rows as number of credits the client had.
*   bureau_balance.csv --> monthly balances of previous credits, one row for each month.
*   POS_CASH_balance.csv --> monthly balance snapshots of previous point of sales and cash loans that the applicant had, one row for each month.
*   credit_card_balance.csv --> monthly balance snapshots of previous credit cards, one row for each month.
*   previous_application.csv --> all previous applications for Home Credit loans of clients who have loans.
*   installments_payments.csv --> repayment history for the previously disbursed credits.

# Load all data into pandas dataframes

In [ ]:
application_train_df = pd.read_csv('application_train.csv')
application_test_df = pd.read_csv('application_test.csv')
bureau_df = pd.read_csv('bureau.csv')
bureau_balance_df = pd.read_csv('bureau_balance.csv')
pos_cash_balance_df = pd.read_csv('POS_CASH_balance.csv')
credit_card_balance_df = pd.read_csv('credit_card_balance.csv')
previous_application_df = pd.read_csv('previous_application.csv')
installments_payments_df = pd.read_csv('installments_payments.csv')

# Function utilities

In [ ]:
def split_columns_by_type(input_df):
  numerical_columns = []
  categorical_columns = []
  for column in application_train_df.columns:
    if application_train_df.dtypes[column] == np.int64 or application_train_df.dtypes[column] == np.float64:
      numerical_columns.append(column)
    else:
      categorical_columns.append(column)
  return numerical_columns, categorical_columns

In [ ]:
def smart_overview(input_df, numerical_columns):
  list_of_numerics = input_df.select_dtypes(include=['float', 'int']).columns
  countUnique = input_df.
  corrSalePrice = round(input_df[numerical_columns].corrwith(input_df['TARGET']), 3) * 100
  types = input_df.dtypes
  missing = round((input_df.isnull().sum()/input_df.shape[0]),3)*100
  overview = input_df.apply(
      lambda x: [
        round(x.min()), 
        round(x.max()), 
        round(x.mean()), 
        round(x.quantile(0.5))
      ] if x.name in list_of_numerics else x.unique())
  outliers = input_df.apply(
      lambda x: sum(
          (x<(x.quantile(0.25)-1.5*(x.quantile(0.75)-x.quantile(0.25)))) | 
          (x>(x.quantile(0.75)+1.5*(x.quantile(0.75)-x.quantile(0.25)))) 
        if x.name in list_of_numerics else ''))
  explore_df = pd.DataFrame({
    'Types': types,
    'CorrSalePrice%': corrSalePrice,
    'Missing%': missing,
    'Overview': overview,
    'Outliers': outliers
  })
  explore_df['Types'] = explore_df['Types'].astype(str)
  return explore_df.transpose()

# Data Exploration

## application_{train|test}.csv

In [ ]:
application_train_columns = list(application_train_df.columns)
application_train_columns.sort()
application_train_df = application_train_df[application_train_columns]

In [ ]:
temp_columns_type = split_columns_by_type(application_train_df)
temp_numerical_columns = temp_columns_type[0]
smart_overview(application_test_df, temp_numerical_columns)

In [ ]:
application_train_df.head(5)

In [ ]:
types = application_train_df.dtypes
types